## Train and deploy with private registry on Kubeflow from Notebooks

- In order to use a private registry in fairing append builder on the Jupyter notebook, two settings are required.
    ```
    # under docker client environ.
    $ docker login <private_registry>
    $ cat ~/.docker/config.json
      
    # in notebook console
    $ mkdir /home/jovyan/.docker
    $ vi /home/jovyan/.docker/config.json
    ```
-  Patch imagePullSecrets on the service account so that the deployer can pull the private registry image.
    ```
    # in current context(or in notebook console)
    $ kubectl create secret generic regcred --from-file=.dockerconfigjson=/home/${USER}/.docker/config.json \
    --type=kubernetes.io/dockerconfigjson -n anonymous  
    $ kubectl patch serviceaccount default -p '{"imagePullSecrets": [{"name": "regcred"}]}' -n anonymous
    ```

In [ ]:
import os
from kubeflow import fairing
import time

In [ ]:
DOCKER_REGISTRY = '<private_registry>'
fairing.config.set_builder('append',base_image='gcr.io/kubeflow-images-public/tensorflow-1.14.0-notebook-cpu:v0.7.0', registry=DOCKER_REGISTRY, push=True)
fairing.config.set_deployer('job')

In [18]:
def train():
    print("hello world!")

In [19]:
# Training on notebook 
train()

hello world!


In [23]:
# Training on current kubernetes context.
remote_train = fairing.config.fn(train)
remote_train()

INFO:root:Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7f7374eb15c0>
INFO:root:Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f7374eb1320>
INFO:root:Using deployer: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f7374eb1320>
INFO:root:Creating docker context: /tmp/fairing_context_if6n8f9a
INFO:root:Loading Docker credentials for repository 'gcr.io/kubeflow-images-public/tensorflow-1.14.0-notebook-cpu:v0.7.0'
INFO:root:Loading Docker credentials for repository 'registry.zipsacoding.com/fairing-job:4DC5EA4'
INFO:root:Layer sha256:107cba84ef3d72ed995c76c7a4f60ba5613f58b029ab7e42ac20ece99bec88b1 exists, skipping
INFO:root:Layer sha256:b5494e32d0131350be270a54399cee65934e90d3c2df87a83757903e627813b2 exists, skipping
INFO:root:Layer sha256:b054a26005b7f3b032577f811421fab5ec3b42ce45a4012dfa00cf6ed6191b0f exists, skipping
INFO:root:Layer sha256:5b7339215d1d5f8e68622d584a224f60339f5bef41dbd

KeyboardInterrupt: 